In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Emotion Analysis\n",
    "\n",
    "This notebook analyzes facial expressions and emotions in classroom video data.\n",
    "\n",
    "## Features:\n",
    "- Emotion detection using various models\n",
    "- Attention analysis (gaze direction, head pose)\n",
    "- Temporal emotion patterns\n",
    "- Student engagement analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import sys\n",
    "sys.path.append('../src')\n",
    "\n",
    "import cv2\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import pandas as pd\n",
    "import seaborn as sns\n",
    "from pathlib import Path\n",
    "import json\n",
    "from collections import Counter\n",
    "\n",
    "from config import Config\n",
    "from detection.face_tracker import FaceTracker\n",
    "from emotion.emotion_detector import EmotionDetector\n",
    "from utils.video_utils import VideoProcessor"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load configuration and initialize components\n",
    "config = Config()\n",
    "video_processor = VideoProcessor(config)\n",
    "face_tracker = FaceTracker(config)\n",
    "emotion_detector = EmotionDetector(config)\n",
    "\n",
    "# Test video path\n",
    "video_path = \"../data/raw_videos/sample_classroom.mp4\"\n",
    "\n",
    "print(\"Components initialized\")\n",
    "print(f\"Video path: {video_path}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Emotion Detection Pipeline"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def process_video_emotions(video_path, max_frames=200):\n",
    "    \"\"\"Process video and extract emotion data.\"\"\"\n",
    "    \n",
    "    cap = cv2.VideoCapture(video_path)\n",
    "    if not cap.isOpened():\n",
    "        print(f\"Failed to open video: {video_path}\")\n",
    "        return []\n",
    "    \n",
    "    all_detections = []\n",
    "    frame_idx = 0\n",
    "    \n",
    "    while frame_idx < max_frames:\n",
    "        ret, frame = cap.read()\n",
    "        if not ret:\n",
    "            break\n",
    "        \n",
    "        # Detect faces\n",
    "        face_detections = face_tracker.detect_faces(frame)\n",
    "        \n",
    "        # Process each face for emotions\n",
    "        for detection in face_detections:\n",
    "            bbox = detection['bbox']\n",
    "            x1, y1, x2, y2 = bbox\n",
    "            \n",
    "            # Extract face region\n",
    "            face_region = frame[y1:y2, x1:x2]\n",
    "            if face_region.size == 0:\n",
    "                continue\n",
    "            \n",
    "            # Detect emotions\n",
    "            emotions = emotion_detector.detect_emotions(face_region)\n",
    "            \n",
    "            # Detect attention\n",
    "            attention = emotion_detector.detect_attention(face_region)\n",
    "            \n",
    "            # Add to detection\n",
    "            detection.update({\n",
    "                'frame_idx': frame_idx,\n",
    "                'emotions': emotions,\n",
    "                'attention': attention,\n",
    "                'dominant_emotion': max(emotions.items(), key=lambda x: x[1])[0],\n",
    "                'emotion_confidence': max(emotions.values()),\n",
    "                'is_attentive': attention['attention_score'] > 0.7\n",
    "            })\n",
    "            \n",
    "            all_detections.append(detection)\n",
    "        \n",
    "        frame_idx += 1\n",
    "        \n",
    "        if frame_idx % 50 == 0:\n",
    "            print(f\"Processed {frame_idx} frames\")\n",
    "    \n",
    "    cap.release()\n",
    "    print(f\"Completed processing {len(all_detections)} detections\")\n",
    "    return all_detections"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Process video for emotion analysis\n",
    "detections = process_video_emotions(video_path)\n",
    "\n",
    "if detections:\n",
    "    print(f\"\\nProcessed {len(detections)} face detections\")\n",
    "    print(f\"Frames analyzed: {max(d['frame_idx'] for d in detections) + 1}\")\n",
    "else:\n",
    "    print(\"No detections found\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Emotion Distribution Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if detections:\n",
    "    # Extract emotion data\n",
    "    emotions = [d['dominant_emotion'] for d in detections]\n",
    "    emotion_counts = Counter(emotions)\n",
    "    \n",
    "    # Create emotion distribution plot\n",
    "    plt.figure(figsize=(12, 6))\n",
    "    \n",
    "    # Bar chart\n",
    "    plt.subplot(1, 2, 1)\n",
    "    emotion_names = list(emotion_counts.keys())\n",
    "    emotion_values = list(emotion_counts.values())\n",
    "    \n",
    "    colors = plt.cm.Set3(np.linspace(0, 1, len(emotion_names)))\n",
    "    bars = plt.bar(emotion_names, emotion_values, color=colors)\n",
    "    plt.title('Emotion Distribution')\n",
    "    plt.xlabel('Emotion')\n",
    "    plt.ylabel('Count')\n",
    "    plt.xticks(rotation=45)\n",
    "    \n",
    "    # Add value labels on bars\n",
    "    for bar, value in zip(bars, emotion_values):\n",
    "        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, \n",
    "                str(value), ha='center', va='bottom')\n",
    "    \n",
    "    # Pie chart\n",
    "    plt.subplot(1, 2, 2)\n",
    "    plt.pie(emotion_values, labels=emotion_names, autopct='%1.1f%%', colors=colors)\n",
    "    plt.title('Emotion Distribution (%)')\n",
    "    \n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "    \n",
    "    # Print statistics\n",
    "    print(\"\\nEmotion Statistics:\")\n",
    "    print(\"-\" * 30)\n",
    "    for emotion, count in emotion_counts.most_common():\n",
    "        percentage = (count / len(emotions)) * 100\n",
    "        print(f\"{emotion.title()}: {count} ({percentage:.1f}%)\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Attention Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if detections:\n",
    "    # Extract attention data\n",
    "    attention_scores = [d['attention']['attention_score'] for d in detections]\n",
    "    head_poses_x = [d['attention']['head_pose_x'] for d in detections]\n",
    "    head_poses_y = [d['attention']['head_pose_y'] for d in detections]\n",
    "    gaze_x = [d['attention']['gaze_x'] for d in detections]\n",
    "    gaze_y = [d['attention']['gaze_y'] for d in detections]\n",
    "    \n",
    "    # Create attention analysis plots\n",
    "    fig, axes = plt.subplots(2, 2, figsize=(15, 10))\n",
    "    \n",
    "    # Attention score distribution\n",
    "    axes[0, 0].hist(attention_scores, bins=20, alpha=0.7, color='skyblue')\n",
    "    axes[0, 0].set_title('Attention Score Distribution')\n",
    "    axes[0, 0].set_xlabel('Attention Score')\n",
    "    axes[0, 0].set_ylabel('Frequency')\n",
    "    axes[0, 0].axvline(np.mean(attention_scores), color='red', linestyle='--', \n",
    "                       label=f'Mean: {np.mean(attention_scores):.3f}')\n",
    "    axes[0, 0].legend()\n",
    "    \n",
    "    # Head pose analysis\n",
    "    axes[0, 1].scatter(head_poses_x, head_poses_y, alpha=0.6, c=attention_scores, cmap='RdYlGn')\n",
    "    axes[0, 1].set_title('Head Pose vs Attention')\n",
    "    axes[0, 1].set_xlabel('Head Pose X (degrees)')\n",
    "    axes[0, 1].set_ylabel('Head Pose Y (degrees)')\n",
    "    axes[0, 1].grid(True, alpha=0.3)\n",
    "    \n",
    "    # Gaze direction\n",
    "    axes[1, 0].scatter(gaze_x, gaze_y, alpha=0.6, c=attention_scores, cmap='RdYlGn')\n",
    "    axes[1, 0].set_title('Gaze Direction vs Attention')\n",
    "    axes[1, 0].set_xlabel('Gaze X')\n",
    "    axes[1, 0].set_ylabel('Gaze Y')\n",
    "    axes[1, 0].grid(True, alpha=0.3)\n",
    "    \n",
    "    # Attention over time\n",
    "    frame_indices = [d['frame_idx'] for d in detections]\n",
    "    axes[1, 1].scatter(frame_indices, attention_scores, alpha=0.6, c=attention_scores, cmap='RdYlGn')\n",
    "    axes[1, 1].set_title('Attention Over Time')\n",
    "    axes[1, 1].set_xlabel('Frame Number')\n",
    "    axes[1, 1].set_ylabel('Attention Score')\n",
    "    axes[1, 1].grid(True, alpha=0.3)\n",
    "    \n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "    \n",
    "    # Print attention statistics\n",
    "    print(\"\\nAttention Statistics:\")\n",
    "    print(\"-\" * 30)\n",
    "    print(f\"Average attention score: {np.mean(attention_scores):.3f}\")\n",
    "    print(f\"Attention score std: {np.std(attention_scores):.3f}\")\n",
    "    print(f\"High attention (>0.7): {sum(1 for s in attention_scores if s > 0.7)} ({sum(1 for s in attention_scores if s > 0.7)/len(attention_scores)*100:.1f}%)\")\n",
    "    print(f\"Low attention (<0.3): {sum(1 for s in attention_scores if s < 0.3)} ({sum(1 for s in attention_scores if s < 0.3)/len(attention_scores)*100:.1f}%)\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Temporal Emotion Patterns"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if detections:\n",
    "    # Create time series data\n",
    "    df = pd.DataFrame(detections)\n",
    "    \n",
    "    # Group by frame and calculate statistics\n",
    "    frame_stats = df.groupby('frame_idx').agg({\n",
    "        'attention': lambda x: np.mean([d['attention_score'] for d in x]),\n",
    "        'emotion_confidence': 'mean',\n",
    "        'dominant_emotion': lambda x: x.mode().iloc[0] if not x.mode().empty else 'neutral'\n",
    "    }).reset_index()\n",
    "    \n",
    "    # Plot temporal patterns\n",
    "    fig, axes = plt.subplots(2, 1, figsize=(15, 10))\n",
    "    \n",
    "    # Attention over time\n",
    "    axes[0].plot(frame_stats['frame_idx'], frame_stats['attention'], 'b-', linewidth=2)\n",
    "    axes[0].set_title('Average Attention Over Time')\n",
    "    axes[0].set_xlabel('Frame Number')\n",
    "    axes[0].set_ylabel('Average Attention Score')\n",
    "    axes[0].grid(True, alpha=0.3)\n",
    "    \n",
    "    # Emotion confidence over time\n",
    "    axes[1].plot(frame_stats['frame_idx'], frame_stats['emotion_confidence'], 'g-', linewidth=2)\n",
    "    axes[1].set_title('Emotion Confidence Over Time')\n",
    "    axes[1].set_xlabel('Frame Number')\n",
    "    axes[1].set_ylabel('Average Emotion Confidence')\n",
    "    axes[1].grid(True, alpha=0.3)\n",
    "    \n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "    \n",
    "    # Emotion transitions\n",
    "    emotion_sequence = frame_stats['dominant_emotion'].tolist()\n",
    "    \n",
    "    # Create emotion transition matrix\n",
    "    emotions = list(set(emotion_sequence))\n",
    "    transition_matrix = np.zeros((len(emotions), len(emotions)))\n",
    "    \n",
    "    for i in range(len(emotion_sequence) - 1):\n",
    "        current_idx = emotions.index(emotion_sequence[i])\n",
    "        next_idx = emotions.index(emotion_sequence[i + 1])\n",
    "        transition_matrix[current_idx, next_idx] += 1\n",
    "    \n",
    "    # Normalize\n",
    "    row_sums = transition_matrix.sum(axis=1)\n",
    "    transition_matrix = transition_matrix / row_sums[:, np.newaxis]\n",
    "    transition_matrix = np.nan_to_num(transition_matrix)\n",
    "    \n",
    "    # Plot transition matrix\n",
    "    plt.figure(figsize=(10, 8))\n",
    "    sns.heatmap(transition_matrix, annot=True, fmt='.2f', \n",
    "                xticklabels=emotions, yticklabels=emotions, cmap='Blues')\n",
    "    plt.title('Emotion Transition Matrix')\n",
    "    plt.xlabel('Next Emotion')\n",
    "    plt.ylabel('Current Emotion')\n",
    "    plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Student Engagement Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if detections:\n",
    "    # Analyze engagement patterns\n",
    "    engagement_data = []\n",
    "    \n",
    "    for detection in detections:\n",
    "        attention = detection['attention']['attention_score']\n",
    "        emotion = detection['dominant_emotion']\n",
    "        confidence = detection['emotion_confidence']\n",
    "        \n",
    "        # Define engagement score\n",
    "        engagement_score = attention * confidence\n",
    "        \n",
    "        # Categorize engagement level\n",
    "        if engagement_score > 0.7:\n",
    "            engagement_level = 'High'\n",
    "        elif engagement_score > 0.4:\n",
    "            engagement_level = 'Medium'\n",
    "        else:\n",
    "            engagement_level = 'Low'\n",
    "        \n",
    "        engagement_data.append({\n",
    "            'frame_idx': detection['frame_idx'],\n",
    "            'attention': attention,\n",
    "            'emotion': emotion,\n",
    "            'confidence': confidence,\n",
    "            'engagement_score': engagement_score,\n",
    "            'engagement_level': engagement_level\n",
    "        })\n",
    "    \n",
    "    # Create engagement analysis\n",
    "    engagement_df = pd.DataFrame(engagement_data)\n",
    "    \n",
    "    # Engagement distribution\n",
    "    plt.figure(figsize=(15, 5))\n",
    "    \n",
    "    plt.subplot(1, 3, 1)\n",
    "    engagement_levels = engagement_df['engagement_level'].value_counts()\n",
    "    colors = ['red', 'orange', 'green']\n",
    "    plt.pie(engagement_levels.values, labels=engagement_levels.index, autopct='%1.1f%%', colors=colors)\n",
    "    plt.title('Engagement Level Distribution')\n",
    "    \n",
    "    plt.subplot(1, 3, 2)\n",
    "    plt.hist(engagement_df['engagement_score'], bins=20, alpha=0.7, color='purple')\n",
    "    plt.title('Engagement Score Distribution')\n",
    "    plt.xlabel('Engagement Score')\n",
    "    plt.ylabel('Frequency')\n",
    "    \n",
    "    plt.subplot(1, 3, 3)\n",
    "    engagement_by_emotion = engagement_df.groupby('emotion')['engagement_score'].mean().sort_values(ascending=False)\n",
    "    engagement_by_emotion.plot(kind='bar', color='lightblue')\n",
    "    plt.title('Average Engagement by Emotion')\n",
    "    plt.xlabel('Emotion')\n",
    "    plt.ylabel('Average Engagement Score')\n",
    "    plt.xticks(rotation=45)\n",
    "    \n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "    \n",
    "    # Print engagement statistics\n",
    "    print(\"\\nEngagement Analysis:\")\n",
    "    print(\"-\" * 30)\n",
    "    print(f\"Average engagement score: {engagement_df['engagement_score'].mean():.3f}\")\n",
    "    print(f\"Engagement score std: {engagement_df['engagement_score'].std():.3f}\")\n",
    "    \n",
    "    print(\"\\nEngagement by level:\")\n",
    "    for level, count in engagement_levels.items():\n",
    "        percentage = (count / len(engagement_df)) * 100\n",
    "        print(f\"  {level}: {count} ({percentage:.1f}%)\")\n",
    "    \n",
    "    print(\"\\nEngagement by emotion:\")\n",
    "    for emotion, score in engagement_by_emotion.items():\n",
    "        print(f\"  {emotion.title()}: {score:.3f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Save Analysis Results"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Save analysis results\n",
    "if detections:\n",
    "    output_path = Path('../data/outputs')\n",
    "    output_path.mkdir(parents=True, exist_ok=True)\n",
    "    \n",
    "    # Save raw detection data\n",
    "    with open(output_path / 'emotion_analysis_results.json', 'w') as f:\n",
    "        json.dump(detections, f, indent=2, default=str)\n",
    "    \n",
    "    # Save summary statistics\n",
    "    summary = {\n",
    "        'total_detections': len(detections),\n",
    "        'frames_analyzed': max(d['frame_idx'] for d in detections) + 1,\n",
    "        'emotion_distribution': dict(Counter(d['dominant_emotion'] for d in detections)),\n",
    "        'average_attention': np.mean([d['attention']['attention_score'] for d in detections]),\n",
    "        'average_emotion_confidence': np.mean([d['emotion_confidence'] for d in detections]),\n",
    "        'engagement_levels': dict(engagement_levels) if 'engagement_levels' in locals() else {}\n",
    "    }\n",
    "    \n",
    "    with open(output_path / 'emotion_analysis_summary.json', 'w') as f:\n",
    "        json.dump(summary, f, indent=2)\n",
    "    \n",
    "    print(f\"Analysis results saved to {output_path}\")\n",
    "    print(f\"  - Raw data: emotion_analysis_results.json\")\n",
    "    print(f\"  - Summary: emotion_analysis_summary.json\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}